# Plotting maps for all 1P parameters (haloID = 0)

In [1]:
pip install regions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 66.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install soxs

Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from regions import RectangleSkyRegion
from astropy import wcs
import soxs
import matplotlib.pyplot as plt

In [42]:
instrument =  "lem_outer_array" #"chandra_acisi_cy0"
sky_center = (0.0, 0.0)
filenames = [
    "1P_1_n5", "1P_1_n4", "1P_1_n3", "1P_1_n2", "1P_1_n1", "1P_1_0", "1P_1_1", "1P_1_2", "1P_1_3", "1P_1_4", "1P_1_5",
    "1P_2_n5", "1P_2_n4", "1P_2_n3", "1P_2_n2", "1P_2_n1", "1P_2_0", "1P_2_1", "1P_2_2", "1P_2_3", "1P_2_4", "1P_2_5",
    "1P_3_n5", "1P_3_n4", "1P_3_n3", "1P_3_n2", "1P_3_n1", "1P_3_0", "1P_3_1", "1P_3_2", "1P_3_3", "1P_3_4", "1P_3_5",
    "1P_4_n5", "1P_4_n4", "1P_4_n3", "1P_4_n2", "1P_4_n1", "1P_4_0", "1P_4_1", "1P_4_2", "1P_4_3", "1P_4_4", "1P_4_5",
    "1P_5_n5", "1P_5_n4", "1P_5_n3", "1P_5_n2", "1P_5_n1", "1P_5_0", "1P_5_1", "1P_5_2", "1P_5_3", "1P_5_4", "1P_5_5",
    "1P_6_n5", "1P_6_n4", "1P_6_n3", "1P_6_n2", "1P_6_n1", "1P_6_0", "1P_6_1", "1P_6_2", "1P_6_3", "1P_6_4", "1P_6_5"
]
print(len(filenames))

66


In [43]:
# Loop over filenames and corresponding numbers (starting from 5)
for number, base in zip(range(0, len(filenames)), filenames):
    halo_str = "000"
    base_path = f"../../Data/X-rays/IllustrisTNG/1P/{base}/snap_032/"
    simput_file = f"IllustrisTNG.1P_{number}.snap_032.halo_{halo_str}.100ks.z0.05.z_simput.fits"
    simput_path = os.path.join(base_path, simput_file)

    if not os.path.exists(simput_path):
        print(f"❌ File not found: {simput_path}")
        continue

    try:
        print(f"🔄 Simulating for {simput_file}...")
        evt_file = f"evt_{number}_{base}.fits"
        img_file = f"img_{number}_{base}.fits"
        plot_file = f"plot_{number}_{base}.png"

        # Run SOXS simulator
        soxs.instrument_simulator(
            simput_path, evt_file, (10.0, "ks"), instrument,
            sky_center, overwrite=True, foreground=False, ptsrc_bkgnd=False
        )

        # Check if there are any events
        with fits.open(evt_file) as f_evt:
            if f_evt[1].data is None or len(f_evt[1].data) == 0:
                print(f"⚠️ No events detected in {evt_file}. Skipping image and plot.")
                continue

        # Write image
        soxs.write_image(evt_file, img_file, emin=0.5, emax=2.0, reblock=1, overwrite=True)

        # Plot image
        center_sky = SkyCoord(*sky_center, unit='deg', frame='fk5')
        region_sky = RectangleSkyRegion(center=center_sky, width=30*u.arcmin, height=30*u.arcmin)

        with fits.open(img_file) as f_img:
            w = wcs.WCS(header=f_img[0].header)
            fig, ax = soxs.plot_image(img_file, stretch='log', cmap='magma', vmax=3000.0, width=0.5)
            ax.add_artist(region_sky.to_pixel(w).as_artist())
            plt.title(f"1P_{number} - {base}")
            plt.savefig(plot_file, dpi=300)
            plt.close()

        print(f"✅ Saved: {plot_file}")

    except Exception as e:
        print(f"❌ Error with 1P_{number} in {base}: {e}")

soxs : [INFO     ] 2025-04-10 02:40:06,161 Making observation of source in evt_0_1P_1_n5.fits.


🔄 Simulating for IllustrisTNG.1P_0.snap_032.halo_000.100ks.z0.05.z_simput.fits...


soxs : [INFO     ] 2025-04-10 02:40:07,622 Detecting events from source IllustrisTNG.1P_0.snap_032.halo_000.100ks.z0.05.z.
soxs : [INFO     ] 2025-04-10 02:40:07,625 Applying energy-dependent effective area from lem_300522.arf.
soxs : [INFO     ] 2025-04-10 02:40:07,696 Pixeling events.
soxs : [INFO     ] 2025-04-10 02:40:07,706 Scattering events with a gaussian-based PSF.
soxs : [INFO     ] 2025-04-10 02:40:07,709 22379 events were detected from the source.
soxs : [INFO     ] 2025-04-10 02:40:07,711 Scattering energies with RMF lem_2.5ev_110422.rmf.


Scattering energies :   0%|          | 0/22379 [00:00<?, ?it/s]

soxs : [INFO     ] 2025-04-10 02:40:10,448 Adding background events.
soxs : [INFO     ] 2025-04-10 02:40:11,799 Adding in instrumental background.
soxs : [INFO     ] 2025-04-10 02:40:11,935 Making 34149 events from the instrumental background.
soxs : [INFO     ] 2025-04-10 02:40:11,945 Writing events to file evt_0_1P_1_n5.fits.
soxs : [INFO     ] 2025-04-10 02:40:12,035 Observation complete.
soxs : [INFO     ] 2025-04-10 02:40:13,722 Making observation of source in evt_1_1P_1_n4.fits.


...

✅ Saved: plot_64_1P_6_4.png
🔄 Simulating for IllustrisTNG.1P_65.snap_032.halo_000.100ks.z0.05.z_simput.fits...


soxs : [INFO     ] 2025-04-10 02:52:19,508 Detecting events from source IllustrisTNG.1P_65.snap_032.halo_000.100ks.z0.05.z.
soxs : [INFO     ] 2025-04-10 02:52:19,510 Applying energy-dependent effective area from lem_300522.arf.
soxs : [INFO     ] 2025-04-10 02:52:19,751 Pixeling events.
soxs : [INFO     ] 2025-04-10 02:52:19,780 Scattering events with a gaussian-based PSF.
soxs : [INFO     ] 2025-04-10 02:52:19,787 77629 events were detected from the source.
soxs : [INFO     ] 2025-04-10 02:52:19,792 Scattering energies with RMF lem_2.5ev_110422.rmf.


Scattering energies :   0%|          | 0/77629 [00:00<?, ?it/s]

soxs : [INFO     ] 2025-04-10 02:52:24,517 Adding background events.
soxs : [INFO     ] 2025-04-10 02:52:25,889 Adding in instrumental background.
soxs : [INFO     ] 2025-04-10 02:52:26,021 Making 34245 events from the instrumental background.
soxs : [INFO     ] 2025-04-10 02:52:26,031 Writing events to file evt_65_1P_6_5.fits.
soxs : [INFO     ] 2025-04-10 02:52:26,155 Observation complete.


✅ Saved: plot_65_1P_6_5.png
